### 2-modes Tic-Tac-Toe with Tkinter
### Easy mode - AI player performing random moves
### Advanced mode - Unbeatable AI player performing moves with Minimax method

In [3]:
## A tic-tac-toe game built with Python and Tkinter.
from tkinter import *
from tkinter import messagebox
import random
import copy

# Initialize global variables
player = 'X'
stop_game = False
ai_mode = "Easy"  # Default to easy

# Game logic
def callback(r, c):
    global player, stop_game

    if stop_game or states[r][c] != 0:
        return  # Ignore further clicks after game over or clicked again on filled cells

    b[r][c].configure(text=player)
    states[r][c] = player

    # Check if it's a win or a draw
    if check_for_winner():
        messagebox.showinfo("Game Over", f"Player {player} wins!")
        stop_game = True
        reset_game()
    elif check_for_draw():
        messagebox.showinfo("Game Over", "It's a Draw!")
        stop_game = True
        reset_game()
    else:
        if player == 'X': # Switch player and call AI
            if ai_mode == "Easy": # Check the selected moode
                random_ai_move()
            else:
                minimax_ai_move()

def check_for_winner():
    # Check rows and columns
    for i in range(3):
        if states[i][0] == states[i][1] == states[i][2] != 0:
            highlight_winner([(i, 0), (i, 1), (i, 2)])
            return True
        if states[0][i] == states[1][i] == states[2][i] != 0:
            highlight_winner([(0, i), (1, i), (2, i)])
            return True
    # Check diagonals
    if states[0][0] == states[1][1] == states[2][2] != 0:
        highlight_winner([(0, 0), (1, 1), (2, 2)])
        return True
    if states[2][0] == states[1][1] == states[0][2] != 0:
        highlight_winner([(2, 0), (1, 1), (0, 2)])
        return True

def check_for_draw(): # Checking if none of the cells are zero
    for row in states:
       if 0 in row:
            return False
    return True

def highlight_winner(cells): # Changes background of winning cells to grey
    for r, c in cells:
        b[r][c].configure(bg='grey')

def reset_game():
    global player, states, stop_game
    # Reset the states, stop_game and player
    states = [[0, 0, 0],
              [0, 0, 0],
              [0, 0, 0]]
    
    stop_game = False
    player = 'X'
    # Reset the board text and color
    for i in range(3):
        for j in range(3):
            b[i][j].configure(text="", bg="black", state="normal",fg="white")

# Random AI player - Easy Mode
import random
def random_ai_move():
    global player, stop_game
    available = [(i, j) for i in range(3) for j in range(3) if states[i][j] == 0]
    if available and not stop_game:
        row, col = random.choice(available)
        b[row][col].configure(text="O",fg="red")
        states[row][col] = 'O'
        if check_for_winner():
            messagebox.showinfo("Game Over", "AI (O) wins!")
            stop_game = True
            reset_game()
        elif check_for_draw():
            messagebox.showinfo("Game Over", "It's a Draw!")
            stop_game = True
            reset_game()
        else:
            player = 'X'  # Human's turn

# Minimax AI player - Advanced Mode
def minimax_ai_move():
    global player, stop_game
    best_score = float('-inf') # Initialize to negative infinity as the score can be 1, 0 or -1.
    best_move = None
    for i in range(3):
        for j in range(3):
            if states[i][j] == 0:
                states[i][j] = 'O' # AI's move
                score = minimax(states, False) # Calling for Player's move score
                states[i][j] = 0
                if score > best_score:
                    best_score = score
                    best_move = (i, j)
    if best_move:
        i, j = best_move
        b[i][j].configure(text="O",fg="red")
        states[i][j] = 'O'
        if check_for_winner():
            messagebox.showinfo("Game Over", "AI (O) wins!")
            stop_game = True
            reset_game()
        elif check_for_draw():
            messagebox.showinfo("Game Over", "It's a Draw!")
            stop_game = True
            reset_game()
        else:
            player = 'X'

# Function to check the minimax score
def minimax(board, is_ai_turn):
    # board - current board states to simulate the moves and get the best move
    # is_ai_turn - True if AI's turn ('O') → wants to maximize score and False if Player's turn ('X') → wants to minimize score
    
    result = evaluate(board)
    if result is not None:
        return result  # +1 for AI win, -1 for player win, 0 for draw

    # AI's turn: maximize the score
    if is_ai_turn:
        best_score = -1000  # a very low starting point
        for i in range(3):
            for j in range(3):
                if board[i][j] == 0:
                    board[i][j] = 'O'  # AI makes a move
                    score = minimax(board, False)  # Now it's player's turn
                    board[i][j] = 0  # undo move
                    best_score = max(best_score, score)
        return best_score
        
    # Player's turn: minimize the score
    else:
        best_score = 1000  # a very high starting point
        for i in range(3):
            for j in range(3):
                if board[i][j] == 0:
                    board[i][j] = 'X'  # Player makes a move
                    score = minimax(board, True)  # Now it's AI's turn
                    board[i][j] = 0  # undo move
                    best_score = min(best_score, score)      
        return best_score
      
# Function to evaluate the board after each simulated move
def evaluate(board):
    for i in range(3):
        if board[i][0] == board[i][1] == board[i][2] != 0:
            return 1 if board[i][0] == 'O' else -1
        if board[0][i] == board[1][i] == board[2][i] != 0:
            return 1 if board[0][i] == 'O' else -1
    # Check for diagonals
    if board[0][0] == board[1][1] == board[2][2] != 0:
        return 1 if board[0][0] == 'O' else -1
    if board[0][2] == board[1][1] == board[2][0] != 0:
        return 1 if board[0][2] == 'O' else -1
    # Check for draw
    if all(cell != 0 for row in board for cell in row):
        return 0
    
    return None # Game is still in progress

# Mode selection
def set_mode(mode):
    global ai_mode
    ai_mode = mode
    mode_prompt.destroy()  # Remove buttons after selection
    messagebox.showinfo("Get Ready!", f"You chose {mode.capitalize()} mode!")

# GUI setup
root = Tk()
root.title("Tic Tac Toe")

# Mode selection prompt
mode_prompt = Toplevel(root)
mode_prompt.attributes('-topmost', True)
mode_prompt.title("Choose Mode")
Label(mode_prompt, text="Are you ready?? Choose the mode", font=("Arial", 14)).pack(pady=10)
Button(mode_prompt, text="Easy", command=lambda: set_mode("Easy"), width=15).pack(pady=5)
Button(mode_prompt, text="Advanced", command=lambda: set_mode("Advanced"), width=15).pack(pady=5)

# Game board setup
b = [[0, 0, 0],
     [0, 0, 0],
     [0, 0, 0]]

states = [[0, 0, 0],
          [0, 0, 0],
          [0, 0, 0]]

for i in range(3):
    for j in range(3):
        b[i][j] = Button(font=("Arial", 56), width=3, bg="black",fg = "white",
                         command=lambda r=i, c=j: callback(r, c))
        b[i][j].grid(row=i, column=j)

player = 'X'
stop_game = False

root.mainloop()